In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re
from webdriver_manager.chrome import ChromeDriverManager  # Importa ChromeDriverManager

# Configura las opciones del navegador
options = Options()
options.add_argument("--headless")  # Si no deseas que se abra una ventana del navegador
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Inicializa el navegador con el ChromeDriver utilizando ChromeDriverManager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Definir la URL base
url_base = 'https://www.paris.cl/ninos/moda/?start={}&sz=24'

# Ruta del archivo Excel
ruta_excel = r"C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\FINALES6.xlsx"

# Intentar cargar el archivo Excel existente (si existe) o crear uno nuevo
try:
    df_existente = pd.read_excel(ruta_excel)
except FileNotFoundError:
    # Si el archivo no existe, se crea un DataFrame vacío con las columnas esperadas
    df_existente = pd.DataFrame(columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])

# Lista para almacenar los productos extraídos
todos_los_productos = []

# Función para extraer los productos de una página específica
def extraer_productos(url):
    driver.get(url)
    
    # Esperamos a que la página cargue completamente (10 segundos de espera máxima)
    wait = WebDriverWait(driver, 10)
    
    try:
        # Esperamos que los productos estén visibles
        productos = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'tw-flex')]//a[@class='tw-w-full tw-h-full tw-cursor-pointer handle-link']")))
        
        if not productos:
            return False

        print(f"Extrayendo productos de la página: {url}")
        nuevos_productos = []

        for index, producto in enumerate(productos, 1):
            try:
                # Extraemos el nombre de la marca
                marca_elemento = producto.find_element(By.XPATH, ".//p[contains(@class, 'tw-text-common-default')]")
                marca = marca_elemento.text if marca_elemento else "Desconocida"

                # Extraemos la información del producto
                nombre_producto = producto.find_element(By.XPATH, ".//p[contains(@class, 'perseo-item-title')]").text
                precio_original = producto.find_element(By.XPATH, ".//h5[contains(@class, 'tw-text-neutral-grey-500')]").text if producto.find_elements(By.XPATH, ".//h5[contains(@class, 'tw-text-neutral-grey-500')]") else ""
                precio_final = producto.find_element(By.XPATH, ".//h3[contains(@class, 'tw-text-common-default')]").text
                url_producto_relativa = producto.get_attribute("href")
                url_producto = f"https://www.paris.cl{url_producto_relativa}"
                sku = url_producto_relativa.split('-')[-1].replace('.html', '')

                # Desplazarse hacia abajo para asegurarse de que todas las imágenes se carguen
                driver.execute_script("window.scrollBy(0, 500);")
                time.sleep(2)  # Aseguramos que las imágenes se carguen

                # Buscar las imágenes
                imagenes_encontradas = producto.find_elements(By.XPATH, ".//img[contains(@src, '.jpg') or contains(@data-src, '.jpg')]")

                url_imagen = ""
                if imagenes_encontradas:
                    url_imagen = imagenes_encontradas[0].get_attribute("src")  # Tomamos la primera imagen que aparece
                    print(f"Imagen encontrada: {url_imagen}")
                else:
                    # Si no se encuentra imagen, revisamos el background-image
                    background_img = producto.get_attribute("style")
                    match = re.search(r'background-image: url\("([^"]+\.jpg)"\)', background_img)
                    if match:
                        url_imagen = match.group(1)
                        print(f"Imagen de fondo encontrada: {url_imagen}")
                
                # Imprimir los datos del producto
                print(f"{index}. Nombre: {nombre_producto}")
                print(f"   Precio Original: {precio_original}")
                print(f"   Precio Final: {precio_final}")
                print(f"   URL Producto: {url_producto}")
                print(f"   SKU: {sku}")
                print(f"   URL Imagen: {url_imagen}")
                print(f"   Marca: {marca}")
                print("-" * 50)
                
                # Guardar los datos del producto
                nuevos_productos.append([nombre_producto, precio_original, precio_final, url_producto, sku, url_imagen, marca])

            except Exception as e:
                print(f"Error al obtener información del producto: {e}")

        if nuevos_productos:
            todos_los_productos.extend(nuevos_productos)  # Agregar los nuevos productos a la lista global

        return True  # Si se han encontrado productos, retornamos True

    except Exception as e:
        print(f"Error al cargar la página {url}: {e}") 
        return False  # Si ocurre algún error, regresamos False

# Empezamos desde la página 1
pagina = 173
while True:
    url = url_base.format(pagina * 24)
    
    if not extraer_productos(url):
        break  # Si no hay productos en esta página, terminamos
    
    pagina += 1

# Crear el DataFrame final con todos los productos y guardarlo en el archivo Excel
df_final = pd.DataFrame(todos_los_productos, columns=["Nombre del Producto", "Precio Original", "Precio Final", "URL del Producto", "SKU", "URL de la Imagen", "Marca"])

# Concatenar los productos extraídos con los productos existentes en el archivo Excel
df_actualizado = pd.concat([df_existente, df_final], ignore_index=True)

# Guardar el DataFrame final en el archivo Excel
df_actualizado.to_excel(ruta_excel, index=False)

# Cierra el navegador
driver.quit()


Extrayendo productos de la página: https://www.paris.cl/ninos/moda/?start=4152&sz=24



KeyboardInterrupt

